In [24]:
import sys
plist = ['/Applications/NEURON/lib/python']
for pd in plist:
    if pd in sys.path:
        sys.path.remove(pd)

In [25]:
# Option 1: NEURON and NMODL defaults
from neuron import h, rxd 

soma = h.Section(name='soma')
cyt = rxd.Region(h.allsec(), name='cyt')
ca = rxd.Species(cyt, name='ca', charge=2)
na = rxd.Species(cyt, name='na', charge=1)
k = rxd.Species(cyt, name='k', charge=1)
unknown = rxd.Species(cyt, name='unknown', charge=-1)

h.finitialize(-65)

1.0

In [27]:
from neuron import h, crxd as rxd

rxd.options.enable.extracellular = True
ecs = rxd.Extracellular(-100, -100, -100,
                        100, 100, 100,
                        dx=20, volume_fraction=0.2, tortuosity=1.6)

ca = rxd.Species(ecs, name='ca', charge=2)
h('cao0_ca_ion = 0.42')
h.finitialize(-65)
print(ca.nodes[0].concentration)

0.42


In [28]:
# Option 2: Uniform initial concentration

soma = h.Section(name='soma')
cyt = rxd.Region([soma], name='cyt')
m = rxd.State(cyt, initial=0.47)

h.finitialize(-65)
print(m.nodes[0].value)

0.47


In [29]:
# Option 3: Initializing to a function of position

from neuron import h, gui, rxd 
h.load_file('stdrun.hoc')
h.load_file('import3d.hoc')

def my_initial(node):
    h.distance(0, h.soma[0](0.5))
    dist = h.distance(node.segment)
    return 2 * h.tanh(dist / 1000)

cell = h.Import3d_SWC_read()
cell.input('c91662.CNG.swc')
h.Import3d_GUI(cell,0)
i3d = h.Import3d_GUI(cell,0)
i3d.instantiate(None)

for sec in h.allsec():
    sec.nseg = 1 + 2 * int(sec.L / 20)

cyt = rxd.Region(h.allsec(), name='cyt', nrn_region='i')
ca = rxd.Species(cyt, name='ca', charge=2, initial=my_initial)
h.finitialize(-65)


IndexError: index 3444 is out of bounds for axis 0 with size 3257

1.0

In [30]:
def my_initial2(node):
    return 1 + h.tanh(node.x3d / 100.)

alpha = rxd.Parameter(cyt, name='alpha', initial=my_initial2)

h.finitialize(-65)

1.0